In [1]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

from streamlit import columns

# Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [2]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [3]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [4]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


Download progress: 100%


## District


In [5]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,11,4200,46200,190000.0,NaN,No,District
1,Ariyalur,Desktop Computer,Article,1,45500,45500,NaN,NaN,No,District
2,Ariyalur,HP Printer 1108 Plus,Article,1,15700,15700,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,16,5200,83200,NaN,-600.0,No,District
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760,23520,165000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
398,Virudhunagar,Ex gratia for Deceased,Aid,1,200000,200000,NaN,NaN,Rameswari,District
399,Virudhunagar,Medical Aid,Aid,1,50000,50000,NaN,NaN,Karuppasamy,District
400,Virudhunagar,Two Wheeler,Article,1,96228,96228,NaN,-225530.0,No,District
401,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372,6372,20000.0,NaN,No,District


In [6]:
district_df.duplicated(subset=["NAME OF THE DISTRICT","REQUESTED ARTICLE","QUANTITY","COMMENTS"]).any()

np.False_

In [7]:
valid_articles(district_df["REQUESTED ARTICLE"])

All articles are valid.


In [8]:
district_df.nunique()

NAME OF THE DISTRICT     63
REQUESTED ARTICLE        84
ITEM TYPE                 3
QUANTITY                 51
COST PER UNIT            89
TOTAL COST              199
ALLOTTED FUNDS           48
EXCESS/SHORTAGE          55
COMMENTS                 65
Beneficiary Type          1
dtype: int64

In [9]:
district_req = district_df.groupby("REQUESTED ARTICLE",as_index=False)["QUANTITY"].sum()
district_req

,REQUESTED ARTICLE,QUANTITY
0,AHUJA Radios,1
1,Agri Battery Sprayer,118
2,Agri Manual Sprayer,33
3,Agri Power Sprayer (2 STK),20
4,Agri Power Sprayer (4 STK),4
...,...,...
79,Wet Grinder 3 Ltrs,1
80,Wet Grinder 5 Ltrs,3
81,Wet Grinder Floor Model 2 Ltr,26
82,Wood Saw Machine (P),2


In [10]:
groupeddt = district_df.groupby("NAME OF THE DISTRICT").agg({
    "TOTAL COST": "sum",
    "ALLOTTED FUNDS": "sum",
    "EXCESS/SHORTAGE": "sum"
}).reset_index()
groupeddt["Check"] = groupeddt["ALLOTTED FUNDS"] -( groupeddt["TOTAL COST"] + groupeddt["EXCESS/SHORTAGE"])
groupeddt

,NAME OF THE DISTRICT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,Check
0,Ariyalur,190600,190000.0,-600.0,0.0
1,Chengalpattu,193120,165000.0,-28120.0,0.0
2,Chennai Central,155560,200000.0,44440.0,0.0
3,Chennai North,252760,250000.0,-2760.0,0.0
4,Chennai South,176612,175000.0,-1612.0,0.0
...,...,...,...,...,...
58,Vellore,672040,675000.0,2960.0,0.0
59,Vijayawada,48464,40000.0,-8464.0,0.0
60,Villupuram,535195,523000.0,-12195.0,0.0
61,Virudhunagar,625530,400000.0,-225530.0,0.0


In [11]:
((district_df["TOTAL COST"] == district_df["COST PER UNIT"]*district_df["QUANTITY"])== False).any()

np.False_

In [12]:
district_df[district_df["REQUESTED ARTICLE"]=="Education Aid"].groupby(["NAME OF THE DISTRICT","TOTAL COST"]).agg({"QUANTITY":"sum"})

QUANTITY
NAME OF THE DISTRICT TOTAL COST          
Chennai Central      15000              1
                     20000              2
Chennai North        15000              1
Chennai South        16000              1
                     22000              1
Cuddalore            80000              4
                     100000             4
Dharmapuri Central   10000              1
                     30000              4
Dharmapuri North     20000              1
Dharmapuri South     50000              1
Dindigul             20000              1
                     40000              4
Kanchipuram          15000              3
Madurai              10000              1
Mumbai, Navi, Pune   15000              1
                     20000              2
Nilagiri             15000              1
                     20000              2
Pudhucheri           25000              1
Pudhukottai          10000              1
Ranipet              32365              1
Thanjavur            77000              1
                     81850              1
                     134100             1
                     174900             1
Theni (R.)           10000              1
Thiruppur            15000              2
Vellore              60000              6
Virudhunagar         75000              1

In [13]:
district_df.groupby(["ITEM TYPE","REQUESTED ARTICLE"])["QUANTITY"].sum()

ITEM TYPE  REQUESTED ARTICLE            
Aid        Business Aid                      5
           Education Aid                    53
           Ex gratia for Deceased            2
           Financial Aid                     2
           Livelihood Aid                    2
                                            ..
Article    Wet Grinder 5 Ltrs                3
           Wet Grinder Floor Model 2 Ltr    26
           Wood Saw Machine (P)              2
           kk.bc.8635.Brush Cutter           2
Project    Project                           1
Name: QUANTITY, Length: 84, dtype: int64

In [14]:
district_df["TOTAL COST"].sum()

np.int64(22742051)

### Public

In [15]:
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,REQUESTED ARTICLE,Cost Per Unit,QUANTITY,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350.0,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,0.0,0,0,Rejected. 15000,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000.0,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000.0,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500.0,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,P 328,S.VASUDEVAN,787000578168,No,"SRI RANGARAJAPURAM, THIRUVANNAMALI",9789102871,Medical Aid,500000.0,1,500000,No,Public,Aid
327,P 329,K.R.GOPALAKRISHNAN,294426843842,No,"OLUPAKKAM,CHENGALPATTU",979038824,Medical Aid,500000.0,1,500000,No,Public,Aid
328,P 330,Brahmin Family,99999,No,Add,9999999999,Groceries,3000.0,5,15000,No,Public,Aid
329,P 331,K.THANGAMANI,571571016351,No,"MELMARUVATHUR, KANCHEEPURAM",9500406779,COW,40000.0,1,40000,No,Public,Article


In [16]:
public_df.to_excel("Public Data.xlsx",index=False)

In [16]:
public_df["REQUESTED ARTICLE"].nunique()

47

In [17]:
valid_articles(public_df["REQUESTED ARTICLE"])

All articles are valid.


In [18]:
public_req = public_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
public_req

,REQUESTED ARTICLE,QUANTITY
0,Agri Battery Sprayer,1
1,Agri Cart Weeder,1
2,Agri Power Sprayer (2 STK),1
3,Aluminium Idli Making Box,1
4,Auto- 3 Wheeler,2
5,Bosch Electrician Kit 13 Re,3
6,Business Aid,33
7,Canon Printer 6030B (USB),1
8,Construction Aid,6
9,Desktop Computer - HVY,1


In [19]:
public_df[(public_df["REQUESTED ARTICLE"]=="Financial Aid")|(public_df["REQUESTED ARTICLE"]=="Business Aid")|(public_df["REQUESTED ARTICLE"]=="Livelihood Aid")|(public_df["REQUESTED ARTICLE"]=="Construction Aid")|(public_df["REQUESTED ARTICLE"]=="Renovation Aid")|(public_df["REQUESTED ARTICLE"]=="Ex gratia for Deceased")&(public_df["QUANTITY"]!=0)]["Total Value"].sum()

np.int64(968000)

In [20]:
public_df[public_df["REQUESTED ARTICLE"]=="Medical Aid"]["Total Value"].sum()

np.int64(1753020)

In [21]:
public_df.groupby(["ITEM TYPE","REQUESTED ARTICLE"])["QUANTITY"].sum()

ITEM TYPE  REQUESTED ARTICLE                    
Aid        Business Aid                              33
           Construction Aid                           6
           Education Aid                             14
           Ex gratia for Deceased                     1
           Financial Aid                             16
           Flood Relief Fund                         10
           Livelihood Aid                             3
           Medical Aid                               18
           Renovation Aid                             8
Article    Agri Battery Sprayer                       1
           Agri Cart Weeder                           1
           Agri Power Sprayer (2 STK)                 1
           Aluminium Idli Making Box                  1
           Auto- 3 Wheeler                            2
           Bosch Electrician Kit 13 Re                3
           Canon Printer 6030B (USB)                  1
           Desktop Computer - HVY                     1

### Institution

In [22]:
inst = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})
inst_df = inst[inst["Beneficiary Type"]=="Institution"]
inst_df

Download progress: 100%


,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE
270,I 001,Cheyyur Thasildar Office,Laptop - i5,1,60800,60800,"Thasildar Office, Cheyyur, Chengalpattu",9999999999,No,Institution,Article
271,I 002,Adhiparasakthi Institutions,Laptop,81,34000,2754000,Melmaruvathur,4427529096,No,Institution,Article
272,I 003,Govt School - MKM,Steel Cupboard,2,12390,24780,Marakanam,9444662668,No,Institution,Article
273,I 004,Govt School - KDPM,Office Table 4X2,5,4602,23010,Kadapakkam,9442826622,No,Institution,Article
274,I 004,Govt School - KDPM,S Type Chair,5,2242,11210,Kadapakkam,9442826622,No,Institution,Article
275,I 005,Govt.School - AVPR,Office Table 4X2,5,4602,23010,"Avanipur, Villupuram",8778403593,No,Institution,Article
276,I 006,Govt.School- NMBR,Steel Cupboard,2,12390,24780,Nolambur,94442375680,No,Institution,Article
277,I 007,Govt.School - CYR,Steel Cupboard,3,12390,37170,Cheiyur,8339068395,No,Institution,Article
278,I 008,Anbu Thondu Nilayam,Rice 1000 Kgs,1,60000,60000,Chengalpattu,9865336966,No,Institution,Article
279,I 009,"Fire Station, Acharapakkam",Epson Printer L3250 (Lite),1,15800,15800,"Acharapakkam, Chengalpattu",9444279203,No,Institution,Article


In [23]:
valid_articles(inst_df["REQUESTED ARTICLE"])

All articles are valid.


In [24]:
inst_df["REQUESTED ARTICLE"].nunique()

24

In [25]:
inst_req = inst_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
inst_req

,REQUESTED ARTICLE,QUANTITY
0,"Acer Monitor 21.5""",15
1,Canon EOS 1500 Camera,1
2,Canon TR 4826 Photocopier,1
3,Ceiling Fan- Ord,35
4,Colour Printer HP Smart Tank 760 All in one,1
5,Desktop Computer,2
6,Epson Printer L3250 (Lite),2
7,Flood Light 50 W,3
8,Foldable Wheel Chair,15
9,Grass Cutter Machine,1


### Others

In [26]:
oth_df = inst[inst["Beneficiary Type"]=="Others"]
oth_df

,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE
0,A 001,Kallakurichi - Kannagi Mayazhagu,Accident Victim,1,3000,3000,Kallakurichi,9999999999,A: 339002257674,Others,Aid
1,A 002,Kallakurichi - Priyanga Gandhi,Accident Victim,1,8000,8000,Kallakurichi,9999999999,A: 545045897495,Others,Aid
2,A 003,Kallakurichi - Sivagami,Accident Victim,1,10000,10000,Kallakurichi,9999999999,A: 290218141057,Others,Aid
3,A 004,Kallakurichi - Rani,Accident Victim,1,7500,7500,Kallakurichi,9999999999,A: 907938031037,Others,Aid
4,A 005,Kallakurichi - Santhi,Accident Victim,1,5000,5000,Kallakurichi,9999999999,A: 906926701129,Others,Aid
...,...,...,...,...,...,...,...,...,...,...,...
265,F 240,Villupuram - VIJAYA MALA.R,Flood Relief Fund,1,7000,7000,Villupuram,9999999999,A: 317038850080,Others,Aid
266,F 241,Villupuram - POONGAVANAM.R,Flood Relief Fund,1,7000,7000,Villupuram,9999999999,A: 666706075923,Others,Aid
267,F 242,Villupuram - CHITHRA.T,Flood Relief Fund,1,7000,7000,Villupuram,9999999999,A: 898411133831,Others,Aid
268,F 243,Villupuram - DHAVAMANI.V,Flood Relief Fund,1,7000,7000,Villupuram,9999999999,A: 671424951882,Others,Aid


In [27]:
valid_articles(oth_df["REQUESTED ARTICLE"])

All articles are valid.


In [28]:
oth_df["REQUESTED ARTICLE"].nunique()

3

In [29]:
oth_req = oth_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
oth_req

,REQUESTED ARTICLE,QUANTITY
0,Accident Victim,21
1,Ex gratia for Deceased,5
2,Flood Relief Fund,244


In [30]:
total_qty = pd.concat([district_req, public_req,inst_req,oth_req]).groupby('REQUESTED ARTICLE', as_index=False).sum()
total_qty

,REQUESTED ARTICLE,QUANTITY
0,AHUJA Radios,1
1,Accident Victim,21
2,"Acer Monitor 21.5""",15
3,Agri Battery Sprayer,119
4,Agri Cart Weeder,1
...,...,...
121,Wet Grinder 3 Ltrs,3
122,Wet Grinder 5 Ltrs,3
123,Wet Grinder Floor Model 2 Ltr,28
124,Wood Saw Machine (P),2


In [31]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
final = pd.concat([district_df,public_df,inst_df]).reset_index(drop=True).groupby(["REQUESTED ARTICLE","Beneficiary Type"], as_index=False)["QUANTITY"].sum()
final

Download progress: 100%
Download progress: 100%
Download progress: 100%


,REQUESTED ARTICLE,Beneficiary Type,QUANTITY
0,AHUJA Radios,District,1
1,Accident Victim,Others,21
2,"Acer Monitor 21.5""",Institution,15
3,Agri Battery Sprayer,District,118
4,Agri Battery Sprayer,Public,1
...,...,...,...
153,Wet Grinder 5 Ltrs,District,3
154,Wet Grinder Floor Model 2 Ltr,District,26
155,Wet Grinder Floor Model 2 Ltr,Public,2
156,Wood Saw Machine (P),District,2


In [32]:
print("District QTY",district_df["QUANTITY"].sum())
print("Public QTY",public_df["QUANTITY"].sum())
print("Inst&Oth QTY",inst_df["QUANTITY"].sum())
print("total qty - merge",final["QUANTITY"].sum())
print("total qty - added",district_df["QUANTITY"].sum() + public_df["QUANTITY"].sum()+inst_df["QUANTITY"].sum())

District QTY 13691
Public QTY 1113
Inst&Oth QTY 480
total qty - merge 15284
total qty - added 15284


In [33]:
# Pivot the DataFrame to create columns for each Beneficiary Type and total
pivot_df = final.pivot_table(index="REQUESTED ARTICLE",columns="Beneficiary Type",values="QUANTITY",aggfunc="sum",fill_value=0).reset_index()

# Add missing columns for Beneficiary Types and calculate the Total
pivot_df["District"] = pivot_df.get("District", 0)
pivot_df["Public"] = pivot_df.get("Public", 0)
pivot_df["Institution"] = pivot_df.get("Institution", 0)
pivot_df["Others"] = pivot_df.get("Others", 0)

pivot_df["Total"] = pivot_df[["District", "Public", "Institution", "Others"]].sum(axis=1)
pivot_df
ord_req_id = "1ou21kOkXQpL-hoaJ-11av2m7Kwk5hsif65jVOiFaU2Y"
existing_data = read_file(ord_req_id)

# Ensure required columns exist
if "Ordered Quantity" not in existing_data:
    existing_data["Ordered Quantity"] = 0
if "Remaining Quantity" not in existing_data:
    existing_data["Remaining Quantity"] = existing_data["Total"]

# Merge existing data with the new ordered quantity
updated_df = pivot_df.merge(
            existing_data[["REQUESTED ARTICLE", "Ordered Quantity", "Remaining Quantity"]],
            on="REQUESTED ARTICLE",
            how="left",
            suffixes=("", "_old")
        ).fillna({"Ordered Quantity": 0, "Remaining Quantity": lambda x: x["Total"]})

updated_df

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Others,Public,Total,Ordered Quantity,Remaining Quantity
0,AHUJA Radios,1,0,0,0,1,1,0
1,Accident Victim,0,0,21,0,21,21,0
2,"Acer Monitor 21.5""",0,15,0,0,15,15,0
3,Agri Battery Sprayer,118,0,0,1,119,125,-6
4,Agri Cart Weeder,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...
121,Wet Grinder 3 Ltrs,1,0,0,2,3,3,0
122,Wet Grinder 5 Ltrs,3,0,0,0,3,3,0
123,Wet Grinder Floor Model 2 Ltr,26,0,0,2,28,28,0
124,Wood Saw Machine (P),2,0,0,0,2,2,0


In [34]:
print("District Total",updated_df["District"].sum())
print("Public Total",updated_df["Public"].sum())
print("Institution Total",updated_df["Institution"].sum())
print("oth Total",updated_df["Others"].sum())


District Total 13691
Public Total 1113
Institution Total 210
oth Total 270


In [35]:
updated_df["Total"].sum()

np.int64(15284)

### Inventory

In [36]:
inventory_df = read_file(ord_req_id)
inventory_df

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Others,Public,Total,Ordered Quantity,Remaining Quantity
0,AHUJA Radios,1,0,0,0,1,1,0
1,Accident Victim,0,0,21,0,21,21,0
2,"Acer Monitor 21.5""",0,15,0,0,15,15,0
3,Agri Battery Sprayer,118,0,0,1,119,125,-6
4,Agri Cart Weeder,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...
122,Wet Grinder 3 Ltrs,1,0,0,2,3,3,0
123,Wet Grinder 5 Ltrs,3,0,0,0,3,3,0
124,Wet Grinder Floor Model 2 Ltr,26,0,0,2,28,28,0
125,Wood Saw Machine (P),2,0,0,0,2,2,0


In [37]:
print("Total after reducing Motors: ",inventory_df["Total"].sum() - inventory_df[inventory_df["REQUESTED ARTICLE"]=="Sewing Motor"]["Total"].sum())
print("Total Sum:", inventory_df["Total"].sum() - inventory_df[inventory_df["REQUESTED ARTICLE"]=="Sewing Motor"]["Total"].sum())
print("Ordered + Remaining Sum:", inventory_df["Ordered Quantity"].sum() + inventory_df["Remaining Quantity"].sum() - inventory_df[inventory_df["REQUESTED ARTICLE"]=="Sewing Motor"]["Total"].sum())

print("**Calculation Check**")
num_mismatches = (inventory_df["Total"] != inventory_df["Ordered Quantity"] + inventory_df["Remaining Quantity"]).sum()
print(f"Number of mismatched rows: {num_mismatches}")

Total after reducing Motors:  15281
Total Sum: 15281
Ordered + Remaining Sum: 15281
**Calculation Check**
Number of mismatched rows: 0


### All records Button

In [38]:
district_df = read_file(master_data_id)
district_df["App. No."]=""
cnt = 1
for names in district_df["NAME OF THE DISTRICT"].unique():
    district_df.loc[district_df["NAME OF THE DISTRICT"] == names, "App. No."] = "D " + str(f"{cnt:03}")
    cnt += 1

df1 = district_df[["App. No.","NAME OF THE DISTRICT","REQUESTED ARTICLE","ITEM TYPE","QUANTITY","COST PER UNIT","TOTAL COST","Beneficiary Type","COMMENTS"]]
df1 = df1.rename(columns={"NAME OF THE DISTRICT":"Name","REQUESTED ARTICLE":"Article Name","COST PER UNIT":"Cost Per Unit","QUANTITY":"Quantity","TOTAL COST":"Total Value","COMMENTS":"Comments"})
df1

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200,46200,District,No
1,D 001,Ariyalur,Desktop Computer,Article,1,45500,45500,District,No
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700,15700,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760,23520,District,No
...,...,...,...,...,...,...,...,...,...
398,D 062,Virudhunagar,Ex gratia for Deceased,Aid,1,200000,200000,District,Rameswari
399,D 062,Virudhunagar,Medical Aid,Aid,1,50000,50000,District,Karuppasamy
400,D 062,Virudhunagar,Two Wheeler,Article,1,96228,96228,District,No
401,D 063,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372,6372,District,No


In [39]:
public_df = read_file(public_master_id)[["App. No.","Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
public_df

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,P 001,Rekha Elumalai,Handicapped Hand Tricycle,Article,1,7350.0,7350,Public,No
1,P 002,G.J.Arulmanimaran,Education Aid,Aid,0,0.0,0,Public,Rejected. 15000
2,P 003,A.Manikandan,Education Aid,Aid,1,15000.0,15000,Public,Rs.15000
3,P 004,V.Jayavel,Business Aid,Aid,1,10000.0,10000,Public,Rs.10000
4,P 005,K.Ganesh,Gents Cycle,Article,1,5500.0,5500,Public,No
...,...,...,...,...,...,...,...,...,...
323,P 325,SUREKA,Education Aid,Aid,1,28298.0,28298,Public,No
324,P 326,J.NARAYANAN,Medical Aid,Aid,1,20000.0,20000,Public,No
325,P 327,D.DHANALAKSHMI,Medical Aid,Aid,1,10000.0,10000,Public,No
326,P 328,S.VASUDEVAN,Medical Aid,Aid,1,500000.0,500000,Public,No


In [40]:
inst_df = read_file(inst_data_id)[["App. No.","Institution Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
df3  = inst_df.rename(columns={"Institution Name":"Name"})
df3

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,A 001,Kallakurichi - Kannagi Mayazhagu,Accident Victim,Aid,1,3000,3000,Others,A: 339002257674
1,A 002,Kallakurichi - Priyanga Gandhi,Accident Victim,Aid,1,8000,8000,Others,A: 545045897495
2,A 003,Kallakurichi - Sivagami,Accident Victim,Aid,1,10000,10000,Others,A: 290218141057
3,A 004,Kallakurichi - Rani,Accident Victim,Aid,1,7500,7500,Others,A: 907938031037
4,A 005,Kallakurichi - Santhi,Accident Victim,Aid,1,5000,5000,Others,A: 906926701129
...,...,...,...,...,...,...,...,...,...
295,I 012,Adhiparasakthi Annai Illam,Podiam Mike- Digimore,Article,1,1900,1900,Institution,No
296,I 012,Adhiparasakthi Annai Illam,Portable Speaker,Article,1,12000,12000,Institution,No
297,I 013,Union Public School,Epson Printer L3250 (Lite),Article,1,15800,15800,Institution,No
298,I 013,Union Public School,Steel Cupboard,Article,1,12390,12390,Institution,No


In [41]:
consold = pd.concat([df1,public_df,df3]).reset_index(drop=True)
consold

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No
...,...,...,...,...,...,...,...,...,...
1026,I 012,Adhiparasakthi Annai Illam,Podiam Mike- Digimore,Article,1,1900.0,1900,Institution,No
1027,I 012,Adhiparasakthi Annai Illam,Portable Speaker,Article,1,12000.0,12000,Institution,No
1028,I 013,Union Public School,Epson Printer L3250 (Lite),Article,1,15800.0,15800,Institution,No
1029,I 013,Union Public School,Steel Cupboard,Article,1,12390.0,12390,Institution,No


In [42]:
consold["Quantity"].sum()

np.int64(15284)

In [43]:
district_df

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type,App. No.
0,Ariyalur,Agri Battery Sprayer,Article,11,4200,46200,190000.0,NaN,No,District,D 001
1,Ariyalur,Desktop Computer,Article,1,45500,45500,NaN,NaN,No,District,D 001
2,Ariyalur,HP Printer 1108 Plus,Article,1,15700,15700,NaN,NaN,No,District,D 001
3,Ariyalur,Sewing Machine ORD,Article,16,5200,83200,NaN,-600.0,No,District,D 001
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760,23520,165000.0,NaN,No,District,D 002
...,...,...,...,...,...,...,...,...,...,...,...
398,Virudhunagar,Ex gratia for Deceased,Aid,1,200000,200000,NaN,NaN,Rameswari,District,D 062
399,Virudhunagar,Medical Aid,Aid,1,50000,50000,NaN,NaN,Karuppasamy,District,D 062
400,Virudhunagar,Two Wheeler,Article,1,96228,96228,NaN,-225530.0,No,District,D 062
401,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372,6372,20000.0,NaN,No,District,D 063


In [44]:
district_df[district_df["TOTAL COST"]==0]

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type,App. No.


In [45]:
fd = district_df[district_df["ITEM TYPE"]=="Article"]
for i in fd["REQUESTED ARTICLE"].unique():
    if fd[fd["REQUESTED ARTICLE"] == i]["COST PER UNIT"].nunique() == 1:
        print("Good")
    else:
        print(i)


Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Two Wheeler
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Provision materials to Orphanages
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good
Good


In [46]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE", "Quantity": "QUANTITY"})[
    ["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name": "REQUESTED ARTICLE", "Quantity": "QUANTITY"})[
    ["REQUESTED ARTICLE", "QUANTITY", "Beneficiary Type"]]
final = pd.concat([district_df, public_df, inst_df]).reset_index(drop=True).groupby(
            ["REQUESTED ARTICLE", "Beneficiary Type"], as_index=False).sum()

Download progress: 100%
Download progress: 100%
Download progress: 100%


In [47]:
public_df = read_file(public_master_id)
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Cost Per Unit,Quantity,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350.0,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,0.0,0,0,Rejected. 15000,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000.0,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000.0,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500.0,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,P 325,SUREKA,862445285117,No,"KOOTHAPAKKAM, CUDDALORE",9443746594,Education Aid,28298.0,1,28298,No,Public,Aid
324,P 326,J.NARAYANAN,470216952282,No,"GINGEE, VILLUPURAM",9898460485,Medical Aid,20000.0,1,20000,No,Public,Aid
325,P 327,D.DHANALAKSHMI,922318597430,No,"IDAYATHUR, CHENGLEPATTU",9944048100,Medical Aid,10000.0,1,10000,No,Public,Aid
326,P 328,S.VASUDEVAN,787000578168,No,"SRI RANGARAJAPURAM, THIRUVANNAMALI",9789102871,Medical Aid,500000.0,1,500000,No,Public,Aid


In [48]:
public_df.groupby(["ITEM TYPE","Article Name"]).count()

App. No.  Name  \
ITEM TYPE Article Name                                            
Aid       Business Aid                                 33    33   
          Construction Aid                              6     6   
          Education Aid                                17    17   
          Ex gratia for Deceased                        1     1   
          Financial Aid                                16    16   
          Flood Relief Fund                            12    12   
          Livelihood Aid                                3     3   
          Medical Aid                                  21    21   
          Renovation Aid                                8     8   
Article   Agri Battery Sprayer                          1     1   
          Agri Cart Weeder                              1     1   
          Agri Power Sprayer (2 STK)                    1     1   
          Aluminium Idli Making Box                     1     1   
          Auto- 3 Wheeler                               2     2   
          Bosch Electrician Kit 13 Re                   3     3   
          Canon Printer 6030B (USB)                     1     1   
          Desktop Computer - HVY                        1     1   
          Dictionary English - Tamil                    1     1   
          Electronic weighing scale                     1     1   
          Fishing net                                   1     1   
          Fruit Juice Mixie                             1     1   
          Gents Cycle                                  12    12   
          Girls Cycle                                  17    17   
          Goat                                          4     4   
          Gravy Grinder 3 Ltr                           1     1   
          Hand Sewing Machine with Motor                2     2   
          Handicapped Hand Tricycle                     5     5   
          Handicapped Scooter                          10    10   
          Hearing Aid                                   2     2   
          Iron Box                                      2     2   
          Junior Cycle                                  6     6   
          Laptop                                       31    31   
          Lenova Tab                                    3     3   
          Mini freezer                                  1     1   
          Push Cart With Top                            7     7   
          Sandalwood tree Sapling                       1     1   
          Sewing Machine ORD / Motor                   63    63   
          TIFFEN SET + MS STOVE 2 BURNER                1     1   
          Table Top Wet Grinder 2 Ltr                   6     6   
          Table fan                                     2     2   
          Tea & Milk Steamer                            1     1   
          Tea can 10 ltrs                               1     1   
          Tiffen Set                                    3     3   
          Tiffen Set + Idli Box +2 burner stove         1     1   
          Two Wheeler                                  10    10   
          Wet Grinder 3 Ltrs                            2     2   
          Wet Grinder Floor Model 2 Ltr                 2     2   

                                                 Aadhar (Without Space)  \
ITEM TYPE Article Name                                                    
Aid       Business Aid                                               33   
          Construction Aid                                            6   
          Education Aid                                              17   
          Ex gratia for Deceased                                      1   
          Financial Aid                                              16   
          Flood Relief Fund                                          12   
          Livelihood Aid                                              3   
          Medical Aid                                                21   
          Renov

In [49]:
public_df.isna().any()

App. No.                  False
Name                      False
Aadhar (Without Space)    False
Handicapped (Yes / No)    False
Address                   False
Mobile                    False
Article Name              False
Cost Per Unit             False
Quantity                  False
Total Value               False
Comments                  False
Beneficiary Type          False
ITEM TYPE                 False
dtype: bool

### Bifurcation of category

In [50]:
ad = articles_df[["Articles", "Category","Sequence List"]].rename(columns={"Articles": "Article Name","Category": "Article Category"})
consold = consold.merge(ad, on="Article Name",how='left')
consold

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments,Article Category,Sequence List
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No,Agriculture,NaN
1,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No,Computers & Printers,NaN
2,D 001,Ariyalur,HP Printer 1108 Plus,Article,1,15700.0,15700,District,No,Computers & Printers,NaN
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No,Sewing Machines,NaN
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No,Agriculture,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1026,I 012,Adhiparasakthi Annai Illam,Podiam Mike- Digimore,Article,1,1900.0,1900,Institution,No,Electronics,NaN
1027,I 012,Adhiparasakthi Annai Illam,Portable Speaker,Article,1,12000.0,12000,Institution,No,Electronics,NaN
1028,I 013,Union Public School,Epson Printer L3250 (Lite),Article,1,15800.0,15800,Institution,No,Computers & Printers,NaN
1029,I 013,Union Public School,Steel Cupboard,Article,1,12390.0,12390,Institution,No,Office Supplies,NaN


In [51]:
consold.groupby(["Article Category","Article Name"]).count()

App. No.  Name  ITEM TYPE  \
Article Category Article Name                                                 
1.Project        Project                                 1     1          1   
2.Aid            Accident Victim                        21    21         21   
                 Business Aid                           38    38         38   
                 Construction Aid                        6     6          6   
                 Education Aid                          49    49         49   
...                                                    ...   ...        ...   
Sewing Machines  Sewing Machine ORD                     37    37         37   
                 Sewing Machine ORD / Motor             73    73         73   
                 Sewing Machine Overlock                 4     4          4   
                 Sewing Machine Universal ZigZag         1     1          1   
Stationery       Dictionary English - Tamil              1     1          1   

                                                  Quantity  Cost Per Unit  \
Article Category Article Name                                               
1.Project        Project                                 1              1   
2.Aid            Accident Victim                        21             21   
                 Business Aid                           38             38   
                 Construction Aid                        6              6   
                 Education Aid                          49             49   
...                                                    ...            ...   
Sewing Machines  Sewing Machine ORD                     37             37   
                 Sewing Machine ORD / Motor             73             73   
                 Sewing Machine Overlock                 4              4   
                 Sewing Machine Universal ZigZag         1              1   
Stationery       Dictionary English - Tamil              1              1   

                                                  Total Value  \
Article Category Article Name                                   
1.Project        Project                                    1   
2.Aid            Accident Victim                           21   
                 Business Aid                              38   
                 Construction Aid                           6   
                 Education Aid                             49   
...                                                       ...   
Sewing Machines  Sewing Machine ORD                        37   
                 Sewing Machine ORD / Motor                73   
                 Sewing Machine Overlock                    4   
                 Sewing Machine Universal ZigZag            1   
Stationery       Dictionary English - Tamil                 1   

                                                  Beneficiary Type  Comments  \
Article Category Article Name                                                  
1.Project        Project                                         1         1   
2.Aid            Accident Victim                                21        21   
                 Business Aid                                   38        38   
                 Construction Aid                                6         6   
                 Education Aid                                  49        49   
...                                                            ...       ...   
Sewing Machines  Sewing Machine ORD                             37        37   
                 Sewing Machine ORD / Motor                     73        73   
                 Sewing Machine Overlock                         4         4   
                 Sewing Machine Universal ZigZag                 1         1   
Stationery       Dictionary English - Tamil                      1         1   

                                                  Sequence List  
Article Category Article Name                                    
1

In [52]:
consold[consold["Article Name"]=="Education Aid"]["Total Value"].sum()

np.int64(1701013)

In [53]:
# grouped = consold.groupby(["Category","Article Name","Name","Beneficiary Type"],as_index=False).agg({"Quantity": "sum", "Total Value":"sum"})
# # grouped["Category"] = grouped["Category"].mask(grouped["Category"].duplicated(), "")
# grouped.to_excel("/Users/aswathshakthi/PycharmProjects/MNP25/Report/Full Report.xlsx",index=False)
# grouped

In [54]:
# grouped["Quantity"].sum()

In [55]:
consold[consold["ITEM TYPE"]=="Aid"]["Quantity"].sum()

np.int64(465)

In [56]:
duplicate_apps = consold.groupby("App. No.")["Name"].nunique() > 1
problem_apps = duplicate_apps[duplicate_apps].index  # App numbers with multiple names

# Show conflicting records
conflicts = consold[consold["App. No."].isin(problem_apps)]
conflicts

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments,Article Category,Sequence List
